# Clustering Consulting Project

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('kmeans-exercise').getOrCreate()

In [3]:
data = spark.read.csv('../data/hack_data.csv', header=True, inferSchema=True)

In [4]:
data.printSchema()
data.describe().show()
data.show(10)

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- hack: integer (nullable = true)

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|              hack|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|     

In [5]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [6]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'hack']

In [7]:
feature_cols = [
    'Session_Connection_Time',
    'Bytes Transferred',
    'Kali_Trace_Used',
    'Servers_Corrupted',
    'Pages_Corrupted',
    'WPM_Typing_Speed',
]

In [8]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

data = assembler.transform(data)

In [9]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

data = scaler.fit(data).transform(data)

In [10]:
data.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|hack|            features|     features_scaled|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|   1|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|   1|[20.0,720.99,0.0,...|[1.41962771166263...|
|                   31.0|           356.32|              1|             3.71|            8.0|            

In [11]:
from pyspark.ml.clustering import KMeans

In [12]:
kmeans2 = KMeans(featuresCol='features_scaled', k=2, seed=42)
kmeans3 = KMeans(featuresCol='features_scaled', k=3, seed=42)

In [13]:
model_k2 = kmeans2.fit(data)
model_k3 = kmeans3.fit(data)

In [14]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [15]:
predicions_2 = model_k2.transform(data)
predicions_3 = model_k3.transform(data)

In [16]:
evaluator = ClusteringEvaluator()

for k, predictions in zip([2, 3], [predicions_2, predicions_3]):
    silhouette = evaluator.evaluate(predictions)
    print(f'Silhouette with squared euclidean distance for k:{k} = {silhouette}')

Silhouette with squared euclidean distance for k:2 = 0.6683623593283755
Silhouette with squared euclidean distance for k:3 = 0.30412315937808737


In [17]:
predicions_2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [18]:
predicions_3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   88|
|         2|   79|
|         0|  167|
+----------+-----+

